In [1]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms, models
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [2]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y).item()
            preds = out.argmax(1)
            ok += (preds == y).sum().item()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in tqdm(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [3]:
augmentation = transforms.RandomAffine(10, scale=(0.9, 1.1), translate=(0.2, 0.2))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])
train_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", 
                            download=True, train=True, transform=transforms.Compose([augmentation, transform]))
valid_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", download=True, train=True, transform=transform)
test_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", download=True, train=False, transform=transform)

train_indices, valid_indices = train_test_split(np.arange(len(train_ds)), test_size=0.2)
train_ds = data.Subset(train_ds, train_indices)
valid_ds = data.Subset(valid_ds, valid_indices)

train_loader = data.DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
valid_loader = data.DataLoader(valid_ds, batch_size=128, shuffle=False, num_workers=4)
test_loader = data.DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=4)

Extracting /tmp/i291318/CIFAR/cifar-10-python.tar.gz to /tmp/i291318/CIFAR
Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
validation(model, valid_loader)

(0.0904, 2.340556829071045)

In [6]:
log_file = open("resnet_log_augmentation_rre.txt", "w")

## Momentum

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-1, k=10, momentum=0.9, weight_decay=1e-5, method="RRE")
loss_fn = nn.NLLLoss()

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.3851



Validation accuracy: 0.3327, validation loss: 1.7670
Epoch 2



Training loss: 1.7340



Validation accuracy: 0.4320, validation loss: 1.5319
Epoch 3


...


Validation accuracy: 0.7738, validation loss: 0.6873
Epoch 29



Training loss: 0.6252



Validation accuracy: 0.7536, validation loss: 0.7357
Epoch 30



Training loss: 0.6101



Validation accuracy: 0.7669, validation loss: 0.6914


In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.790725, 0.5993526573181153)
Valid: (0.7669, 0.6914285528182983)


In [11]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [12]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.7878, 0.6102677832603455)
Valid: (0.7641, 0.7042394050598144)


In [13]:
optimizer.param_groups[0]["lr"] = 1e-2

In [14]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.5187



Validation accuracy: 0.7951, validation loss: 0.6240
Epoch 2



Training loss: 0.4882



Validation accuracy: 0.7940, validation loss: 0.6221
Epoch 3


...


Validation accuracy: 0.8081, validation loss: 0.6333
Epoch 29



Training loss: 0.3575



Validation accuracy: 0.8065, validation loss: 0.6552
Epoch 30



Training loss: 0.3528



Validation accuracy: 0.8065, validation loss: 0.6519


In [15]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.884575, 0.32031138463020326)
Valid: (0.8065, 0.6519221453666687)


In [16]:
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [17]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.882225, 0.3325013121128082)
Valid: (0.8112, 0.6401783519744874)


In [18]:
optimizer.param_groups[0]["lr"] = 1e-3

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.3421



Validation accuracy: 0.8095, validation loss: 0.6334
Epoch 2



Training loss: 0.3390



Validation accuracy: 0.8104, validation loss: 0.6361
Epoch 3


...


Validation accuracy: 0.8098, validation loss: 0.6556
Epoch 29



Training loss: 0.3207



Validation accuracy: 0.8100, validation loss: 0.6559
Epoch 30



Training loss: 0.3182



Validation accuracy: 0.8095, validation loss: 0.6677


In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.894325, 0.29923148522377013)
Valid: (0.8095, 0.6676879231452942)


In [21]:
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [22]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.891325, 0.30091053719520566)
Valid: (0.8088, 0.6652360146999359)
